In [75]:
import yfinance as yf
import smtplib
from datetime import datetime
from dateutil.relativedelta import relativedelta  
import schedule

In [149]:
# gather data from yahoo finance

def load_data(ticker_name):
    
    # set parameters
    start = (datetime.today()-relativedelta(years = 1)).strftime('%Y-%m-%d')
    end = datetime.today().strftime('%Y-%m-%d')
    ticker_names = [ticker_name]
    
    # collect data
    ticker_dataframes = [yf.download(current_tname, start, end) for current_tname in ticker_names]
    ticker_data = dict(zip(ticker_names, ticker_dataframes))
    for name, data in ticker_data.items():
        sma_200 = data[["Close"]].rolling(window=200).mean()
        close_df = data[["Close"]]

In [150]:
# notifies stock updates through email

def notify(email_text, addresses):
    
    # user
    gmail_user = <put your gmail address without @gmail.com>
    gmail_password = <your gmail password>
    
    # email properties
    sent_from = gmail_user
    subject = "Stock updates"
    message = 'Subject: {}\n\n{}'.format(subject, email_text)
    
    #email send request
    try:
        server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
        server.ehlo()
        server.login(gmail_user, gmail_password)
        server.sendmail(sent_from, addresses, message)
        server.close()
        print ('Email sent!')
        
    except Exception as e:
        print(e)
        print ('Something went wrong...')

In [151]:
# runs the notify() function only when last close price is less than 200-day SMA

def monitor_and_send(ticker_names, addresses):
    
    # gather ticker names whose price is lower than 200 SMA
    news = []
    for ticker_name in ticker_names:
        load_data(ticker_name)
        if close_df["Close"][-1] < sma_200["Close"][-1]:
            news.append(ticker_name)
    news = ','.join(news) 
    
    # construct email text and execute email
    email_text = """Yesterday's close price was lower than the 200-day SMA for: {}""".format(news)
    notify(email_text, addresses)

In [ ]:
# sends the email update every day at 8 am

ticker_names = ["VTI, SPY, WMT"]
email_list = ["joeomatoi2023@u.northwestern.edu", "awestin@iu.edu"]

def run():
    monitor_and_send(ticker_names, email_list)
      
schedule.every().day.at("8:00").do(run)
 
while True:
    schedule.run_pending()